# map()

- map() 함수는 iterable한 data type을 입력으로 받는 함수이다.
```python
map(function,iterable datatype)
```
- list, dict와 같이, Series도 iterable한 data type이다.
- map()을 사용하면, 인자로 넣은 function을 data type의 모든 요소에 적용한 값을 return한다.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Series는 하나의 column으로 이루어진 data type이므로 map()을 사용할 수 있다.
info = pd.DataFrame({
    'id' : ['곽홍재','김보현','신지섭','이소연'],
    '학번' : ['17학번','18학번','14학번','18학번'],
    '좋아하는것' : ['컴퓨터','떡볶이','밥','과자']
})
info

,id,학번,좋아하는것
0,곽홍재,17학번,컴퓨터
1,김보현,18학번,떡볶이
2,신지섭,14학번,밥
3,이소연,18학번,과자


In [3]:
list(map(lambda x : x[:2], info['학번']))

['17', '18', '14', '18']

In [4]:
# 하지만 DataFrame으로 map()함수를 쓸 경우, 제대로 인식되지 않는다.
# 이때는 column name을 Series로 생각해서 인식한다.
list(map(lambda x : x, info))

['id', '학번', '좋아하는것']

# Apply

- apply는 dataframe의 row 또는 column들에게 똑같은 function을 일괄적으로 적용시켜주는 방법이다.
```python
dataframe.apply(function)
```

In [5]:
# Series에도 적용 가능하다.
pd.Series(range(1,5)).apply(lambda x: 1/x)

0    1.000000
1    0.500000
2    0.333333
3    0.250000
dtype: float64

In [6]:
# Dataframe에서 사용할 경우, 특정 column값을 지정해주지 않으면 모든 값에 대하여 apply한다.
tr = pd.DataFrame({
        'id': [1,1,1,1,2,2,2],
        'site': ['a','b','c','a','a','b','b'],
        'pageview': np.arange(1,8),
        'dwelltime': np.arange(7.0, 0, -1),
    }, columns=['id','site','pageview','dwelltime'])
tr

,id,site,pageview,dwelltime
0,1,a,1,7.0
1,1,b,2,6.0
2,1,c,3,5.0
3,1,a,4,4.0
4,2,a,5,3.0
5,2,b,6,2.0
6,2,b,7,1.0


In [7]:
tr.apply(lambda x : x.count())

id           7
site         7
pageview     7
dwelltime    7
dtype: int64

In [8]:
# 따라서, 특정 자료형(int)에서만 사용 가능한 함수를 쓰면 에러가 뜬다.
tr.apply(lambda x : 1/x)

TypeError: ("unsupported operand type(s) for /: 'int' and 'str'", 'occurred at index site')

In [9]:
# 이러한 경우, 특정 column만 indexing하여 apply해주자.
tr[['pageview','dwelltime']].apply(lambda x: 1/x)

,pageview,dwelltime
0,1.000000,0.142857
1,0.500000,0.166667
2,0.333333,0.200000
3,0.250000,0.250000
4,0.200000,0.333333
5,0.166667,0.500000
6,0.142857,1.000000


In [10]:
# def로 함수를 만든 경우, 각 함수의 인자를 전달하기 위하여 apply() 안에 args() 인자를 넣어줄 수 있다.
def normalize(x, min_x, max_x): 
    return (x-min_x)/(max_x-min_x)

min_x = tr['pageview'].min(); max_x = tr['pageview'].max()
print(min_x, max_x)

1 7


In [11]:
tr['pageview'].apply(normalize, args=(min_x, max_x))

0    0.000000
1    0.166667
2    0.333333
3    0.500000
4    0.666667
5    0.833333
6    1.000000
Name: pageview, dtype: float64

In [13]:
# axis값을 지정해주지 않을 경우, 기본적으로 column을 기준으로 연산한다.
np.random.seed(777)
frame = pd.DataFrame(np.random.randn(4,3), columns=list('abc'))
frame

,a,b,c
0,-0.468209,-0.822825,-0.065380
1,-0.713362,0.906351,0.766237
2,0.826054,-1.323683,-1.752445
3,1.002449,0.544809,1.895161


In [14]:
frame.apply(lambda x: x.max() - x.min()) ## 각각 하나의 column별로 연산이 된다.

a    1.715811
b    2.230034
c    3.647605
dtype: float64

In [15]:
# axis = 1을 넣어주면, row을 기준으로 연산한다.
frame.apply(lambda x: x.max() - x.min(), axis=1)

0    0.757445
1    1.619713
2    2.578499
3    1.350351
dtype: float64

In [16]:
# applymap 메소드는 apply처럼 각 row(axis=1)나 각 column(axis=0)별로 작동하는 것이 아니라, 각 요소(element)별로 작동한다.
# 따라서 apply와는 함수가 살짝 달라야 한다.
frame.applymap(np.square)

,a,b,c
0,0.219219,0.677041,0.004275
1,0.508885,0.821472,0.587119
2,0.682365,1.752136,3.071062
3,1.004904,0.296817,3.591635


In [17]:
# pandas의 map 메소드는 아래와 같이 어떤 값을 치환할 때 유용하게 사용할 수 있다.
frame['target'] = ['Y','N','Y','N'] # 새로운 column 추가
frame['target'] = frame['target'].map({'Y':1, 'N':0})
frame

,a,b,c,target
0,-0.468209,-0.822825,-0.065380,1
1,-0.713362,0.906351,0.766237,0
2,0.826054,-1.323683,-1.752445,1
3,1.002449,0.544809,1.895161,0


# Datetime

In [18]:
# datetime이라는 자료형은 따로 만들어줘야 한다.
# 기본적으로 이러한 자료형은 string으로 인식한다.
example = pd.Series(['2018-04-16', '2018-04-17', '2018-04-18', '2018-04-19', '2018-04-20'])
example

0    2018-04-16
1    2018-04-17
2    2018-04-18
3    2018-04-19
4    2018-04-20
dtype: object

In [19]:
# to_datetime이라는 메소드를 사용하면, 일정 규격에 맞는 string을 datetime type으로 바꿔준다.
t = pd.to_datetime(example)
t

0   2018-04-16
1   2018-04-17
2   2018-04-18
3   2018-04-19
4   2018-04-20
dtype: datetime64[ns]

In [20]:
# datetime은 연, 월, 일, 요일 등을 뽑아낼 수 있다.
t[0].year , t[0].month, t[0].day, t[0].weekday()

(2018, 4, 16, 0)

In [21]:
# 간단하게 data.dt.속성 형식으로 시계열 데이터를 다룰 수 있다.
t.dt.year, t.dt.month, t.dt.day, t.dt.dayofweek

(0    2018
 1    2018
 2    2018
 3    2018
 4    2018
 dtype: int64, 0    4
 1    4
 2    4
 3    4
 4    4
 dtype: int64, 0    16
 1    17
 2    18
 3    19
 4    20
 dtype: int64, 0    0
 1    1
 2    2
 3    3
 4    4
 dtype: int64)

In [22]:
# 요일은 숫자형태로 주어지는데, 이를 day_name()이라는 함수를 통해 문자로 바꿀 수 있다.
t.dt.day_name()

0       Monday
1      Tuesday
2    Wednesday
3     Thursday
4       Friday
dtype: object